# Query completion

In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam

2024-05-13 16:24:26.765202: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 16:24:26.765362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 16:24:26.964991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 16:24:27.489281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 16:24:32.507437: W tensorflow/compiler/tf2

In [2]:
# Read queries from the json file
queries_json = json.load(open( "cranfield/cran_queries.json", 'r'))[:]
query_ids, queries = [item["query number"] for item in queries_json], \
                        [item["query"] for item in queries_json]


In [3]:
# queries_sent
query_merged = ''.join(queries)
queries_sent = query_merged.split('.')
queries_sent

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft ',
 'what problems of heat conduction in composite slabs have been solved so far ',
 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium ',
 'what chemical kinetic system is applicable to hypersonic aerodynamic problems ',
 'what theoretical and experimental guides do we have as to turbulent couette flow behaviour ',
 'is it possible to relate the available pressure distributions for an ogive forebody at zero angle of attack to the lower surface pressures of an equivalent ogive forebody at angle of attack ',
 'what methods -dash exact or approximate -dash are presently available for predicting body pressures at angle of attack',
 'papers on 

## using a neural network model

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(queries_sent)
print(tokenizer.word_index)

{'the': 1, 'of': 2, 'a': 3, 'in': 4, 'what': 5, 'on': 6, 'to': 7, 'is': 8, 'for': 9, 'flow': 10, 'and': 11, 'are': 12, 'with': 13, 'be': 14, 'can': 15, 'boundary': 16, 'how': 17, 'has': 18, 'buckling': 19, 'an': 20, 'at': 21, 'been': 22, 'pressure': 23, 'available': 24, 'layer': 25, 'effect': 26, 'heat': 27, 'effects': 28, 'hypersonic': 29, 'transfer': 30, 'by': 31, 'theory': 32, 'have': 33, 'body': 34, 'problem': 35, 'anyone': 36, 'papers': 37, 'experimental': 38, 'shells': 39, 'aerodynamic': 40, 'transonic': 41, 'there': 42, 'which': 43, 'cylindrical': 44, 'investigated': 45, 'do': 46, 'it': 47, 'methods': 48, 'does': 49, 'under': 50, 'cylinders': 51, 'solutions': 52, 'theoretical': 53, 'as': 54, 'possible': 55, 'that': 56, 'linear': 57, 'any': 58, 'flutter': 59, 'non': 60, 'analytical': 61, 'shock': 62, 'results': 63, 'flows': 64, 'data': 65, 'made': 66, 'wing': 67, 'viscous': 68, 'method': 69, 'high': 70, 'surface': 71, 'or': 72, 'over': 73, 'laminar': 74, 'interference': 75, 'low'

In [5]:
input_sequences = []
for query in queries_sent:
    sequence = tokenizer.texts_to_sequences([query])[0]
    for i in range(1,len(sequence)):
        input_sequences.append(sequence[:i+1])
        
# pad sequences to equal length
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding = 'pre'))

# create predictors and labels
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]

total_words = len(tokenizer.word_index)+1
ys = tf.keras.utils.to_categorical(labels, num_classes = total_words)
    

In [6]:
model = Sequential()
model.add(Embedding(total_words, 16, input_length = max_seq_len -1))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(total_words, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

2024-05-13 16:24:37.457136: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 48, 16)            15312     
                                                                 
 bidirectional (Bidirection  (None, 40)                5920      
 al)                                                             
                                                                 
 dense (Dense)               (None, 957)               39237     
                                                                 
Total params: 60469 (236.21 KB)
Trainable params: 60469 (236.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
history = model.fit(xs, ys, epochs = 500, verbose = 1)

Epoch 1/500
115/115 [==============================] - 4s 12ms/step - loss: 6.3486 - accuracy: 0.0628
Epoch 2/500
115/115 [==============================] - 1s 12ms/step - loss: 5.8399 - accuracy: 0.0658
Epoch 3/500
115/115 [==============================] - 1s 12ms/step - loss: 5.7908 - accuracy: 0.0658
Epoch 4/500
115/115 [==============================] - 1s 12ms/step - loss: 5.7333 - accuracy: 0.0675
Epoch 5/500
115/115 [==============================] - 1s 12ms/step - loss: 5.6807 - accuracy: 0.0713
Epoch 6/500
115/115 [==============================] - 1s 12ms/step - loss: 5.6336 - accuracy: 0.0691
Epoch 7/500
115/115 [==============================] - 1s 12ms/step - loss: 5.5894 - accuracy: 0.0784
Epoch 8/500
115/115 [==============================] - 1s 12ms/step - loss: 5.5509 - accuracy: 0.0765
Epoch 9/500
115/115 [==============================] - 1s 12ms/step - loss: 5.5136 - accuracy: 0.0915
Epoch 10/500
115/115 [==============================] - 1s 11ms/step - loss: 5.474

In [8]:
# to convert number to string associated with the label
reverse_word_index = {val : key for (key,val) in tokenizer.word_index.items()}
reverse_word_index

{1: 'the',
 2: 'of',
 3: 'a',
 4: 'in',
 5: 'what',
 6: 'on',
 7: 'to',
 8: 'is',
 9: 'for',
 10: 'flow',
 11: 'and',
 12: 'are',
 13: 'with',
 14: 'be',
 15: 'can',
 16: 'boundary',
 17: 'how',
 18: 'has',
 19: 'buckling',
 20: 'an',
 21: 'at',
 22: 'been',
 23: 'pressure',
 24: 'available',
 25: 'layer',
 26: 'effect',
 27: 'heat',
 28: 'effects',
 29: 'hypersonic',
 30: 'transfer',
 31: 'by',
 32: 'theory',
 33: 'have',
 34: 'body',
 35: 'problem',
 36: 'anyone',
 37: 'papers',
 38: 'experimental',
 39: 'shells',
 40: 'aerodynamic',
 41: 'transonic',
 42: 'there',
 43: 'which',
 44: 'cylindrical',
 45: 'investigated',
 46: 'do',
 47: 'it',
 48: 'methods',
 49: 'does',
 50: 'under',
 51: 'cylinders',
 52: 'solutions',
 53: 'theoretical',
 54: 'as',
 55: 'possible',
 56: 'that',
 57: 'linear',
 58: 'any',
 59: 'flutter',
 60: 'non',
 61: 'analytical',
 62: 'shock',
 63: 'results',
 64: 'flows',
 65: 'data',
 66: 'made',
 67: 'wing',
 68: 'viscous',
 69: 'method',
 70: 'high',
 71: 'su

In [9]:
import numpy as np

def complete_query(incomplete_query, next_n_words=1):
    seed_text = incomplete_query
    next_words = next_n_words
    
    completed_query = seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')
        predicted_probabilities = model.predict(token_list, verbose=0)
        predicted_class = np.argmax(predicted_probabilities)
        out_word = reverse_word_index[predicted_class]
        seed_text += " " + out_word
        completed_query += " " + out_word
        
    return completed_query
print(complete_query("experimental studies of creep",50),'\n')
print(complete_query("experimental studies of creep",74))

experimental studies of creep buckling of revolution at angle of revolution on a body of revolution in a body of realistic effects with relative simplicity of the buckling surface lifting function buckling general theory theory theory theory wave wave temperature limit number cylindrical deformations accomplished similitude tumbling possibility strong sensitive subjected of stop outer 

experimental studies of creep buckling of revolution at angle of revolution on a body of revolution in a body of realistic effects with relative simplicity of the buckling surface lifting function buckling general theory theory theory theory wave wave temperature limit number cylindrical deformations accomplished similitude tumbling possibility strong sensitive subjected of stop outer tumbling presently presently define invert planform among studies sectional work simplified neglecting input transformation significance coefficients ahead wake tumbling magnitude kink transition wave made
